# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42513,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:39951,Total threads: 4
Dashboard: http://127.0.0.1:33275/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:46085,


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,980,Zelda,-0.663621,0.041576
1,944,Sarah,-0.417104,-0.774576
2,998,Xavier,-0.381913,-0.798817
3,995,George,-0.177612,-0.580905
4,992,Edith,-0.896568,-0.269685


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,1.000052e+03,-3.411067e-04,-1.174631e-04
std,3.163257e+01,5.774526e-01,5.773556e-01
min,8.530000e+02,-9.999994e-01,-9.999999e-01
25%,9.790000e+02,-4.954523e-01,-4.963042e-01
50%,1.000000e+03,7.686267e-03,5.961936e-03
75%,1.022000e+03,5.057650e-01,5.041535e-01
max,1.155000e+03,9.999995e-01,9.999991e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
979,Ray,0.889312393179893
970,Hannah,0.631410267182476
955,Yvonne,-0.3958932082006319
1006,Patricia,-0.4538264586839879
990,Norbert,0.745250510293965
976,Frank,0.2343389594815179
1041,Michael,0.1460445631992171
1013,Zelda,-0.8030946715491667
1085,Patricia,0.9619572894664176


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,980,Zelda,-0.663621,0.041576
1,944,Sarah,-0.417104,-0.774576
2,998,Xavier,-0.381913,-0.798817
3,995,George,-0.177612,-0.580905
4,992,Edith,-0.896568,-0.269685


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,980,Zelda,-0.663621,0.041576
1,944,Sarah,-0.417104,-0.774576
2,998,Xavier,-0.381913,-0.798817
3,995,George,-0.177612,-0.580905
4,992,Edith,-0.896568,-0.269685


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,979,Ray,0.889312,NaN
1,970,Hannah,0.631410,NaN
2,955,Yvonne,-0.395893,NaN
3,1006,Patricia,-0.453826,NaN
4,990,Norbert,0.745251,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-e79fc90f-2004-4e28-850b-19e90f66d86f')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,980,Zelda,-0.663621,0.041576
1,944,Sarah,-0.417104,-0.774576
2,998,Xavier,-0.381913,-0.798817
3,995,George,-0.177612,-0.580905
4,992,Edith,-0.896568,-0.269685
...,...,...,...,...
86395,1078,Michael,-0.785045,0.003417
86396,993,Ingrid,-0.242918,0.963165
86397,1018,Oliver,0.106002,0.696521
86398,980,Ursula,-0.199759,0.912043


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-25.csv',
 'data/2000-01-20.csv',
 'data/2000-01-29.csv',
 'data/2000-01-02.csv',
 'data/2000-01-19.csv',
 'data/2000-01-23.csv',
 'data/2000-01-10.csv',
 'data/2000-01-21.csv',
 'data/2000-01-17.csv',
 'data/2000-01-04.csv',
 'data/2000-01-27.csv',
 'data/2000-01-22.csv',
 'data/2000-01-14.csv',
 'data/2000-01-11.csv',
 'data/2000-01-13.csv',
 'data/2000-01-08.csv',
 'data/2000-01-09.csv',
 'data/2000-01-06.csv',
 'data/2000-01-01.csv',
 'data/2000-01-07.csv',
 'data/2000-01-12.csv',
 'data/2000-01-16.csv',
 'data/2000-01-26.csv',
 'data/2000-01-24.csv',
 'data/2000-01-18.csv',
 'data/2000-01-15.csv',
 'data/2000-01-03.csv',
 'data/2000-01-30.csv',
 'data/2000-01-28.csv',
 'data/2000-01-05.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-cfab6c78-c6f8-44b3-b788-40fa47fbf5c5'),
 Delayed('read_data-fb1f9aef-4ed3-4238-ad2b-4faa1ff75de5'),
 Delayed('read_data-770df088-49ce-40db-a2c1-3f2cff24a39d'),
 Delayed('read_data-c344be29-0709-4ac4-827b-13a856f17bef'),
 Delayed('read_data-e56909d9-849a-49c9-b28b-5813818ae539'),
 Delayed('read_data-e5fc840b-b06b-412b-b898-5c9f32cb99d3'),
 Delayed('read_data-04529919-5c80-4ba6-90bb-a49981286f35'),
 Delayed('read_data-691c6ebe-f440-4be9-9efa-e03e4c0f7e64'),
 Delayed('read_data-4e8557a7-5d77-4b52-9766-097c1fe7b60f'),
 Delayed('read_data-6b355131-c380-41dd-881a-40d036a2d914'),
 Delayed('read_data-07f47857-f5cd-44a3-bc85-89e3184e11fc'),
 Delayed('read_data-e9e38fae-7ec1-4476-ad2b-19e7ef824799'),
 Delayed('read_data-1e724830-a78d-4052-8e75-973232860352'),
 Delayed('read_data-254b32a0-9af5-412d-a86c-fdb23e04bb35'),
 Delayed('read_data-dcd150bb-70a2-4b61-8107-5e420675bb5d'),
 Delayed('read_data-6583aced-de2a-44ec-ac03-06545228104b'),
 Delayed('read_data-f8e03a57-2b10-49a9-b

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1039,Hannah,-0.898445,0.363494
1,1039,Jerry,0.968003,-0.096370
2,1006,Quinn,-0.668001,-0.505064
3,1011,Dan,0.179424,-0.756545
4,997,Ursula,-0.277329,-0.915346


In [26]:
df.describe().compute()

,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,1.000048e+03,-3.581842e-04,-2.097954e-04
std,3.162779e+01,5.774703e-01,5.773785e-01
min,8.530000e+02,-9.999994e-01,-9.999996e-01
25%,9.790000e+02,-4.954523e-01,-4.963042e-01
50%,1.000000e+03,7.686267e-03,5.961936e-03
75%,1.022000e+03,5.057650e-01,5.041535e-01
max,1.149000e+03,9.999995e-01,9.999991e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.